In [84]:
import os
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain.prompts import PromptTemplate
# Run chain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [85]:
os.environ["token"] = "Bearer token|123675a6-95f6-4fb7-bc95-30095472ae3a|02de311fd83421a7fd637bf34dc8f959caa29f39888d2919e4b9640a2220224b"
token = os.environ["token"]

In [86]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.lab45.ai/v1.1/skills/completion/query'
    
    backend:        Optional[str]   = 'gpt-35-turbo-16k'
    temp:           Optional[float] = 0.7
    top_p:          Optional[float] = 0.1
    top_k:          Optional[int]   = 40
    n_batch:        Optional[int]   = 8
    n_threads:      Optional[int]   = 4
    n_predict:      Optional[int]   = 256
    max_tokens:     Optional[int]   = 256
    repeat_last_n:  Optional[int]   = 64
    repeat_penalty: Optional[float] = 1.18

    class Config:
        extra = Extra.forbid

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    @property
    def _get_model_default_parameters(self):
        return {
            "max_tokens": self.max_tokens,
            #"n_predict": self.n_predict,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            #"n_batch": self.n_batch,
            #"repeat_penalty": self.repeat_penalty,
            #"repeat_last_n": self.repeat_last_n,
        }

    def _call(
        self,
        prompt: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": prompt,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)
        response.raise_for_status()

        # print("API Response:", response.json())

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url,
            'model_parameters': self._get_model_default_parameters
            }

In [87]:
llm = LlamaLLM()

In [88]:
skill1="Core Java L1"
skill2="Spring Boot L2"
skill3="Hibernate L3"
skill4="Selenium L4"
user = "system"
prompt = """As a skilled Technical Interview Panel with advanced knowledge in technology and data science, your role is to meticulously evaluate a employer job description and provide detailed job description having below points outlined.
            Use the mentioned skills to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
            1. **Job Title and Summary**:
            - "What are the key responsibilities?"
            
            2. **Key Responsibilities**:
            - "List the main duties and responsibilities."
            - "What are the daily tasks involved?"

            3. **Qualifications and Skills**:
            - "What qualifications are required?"
            - "List the essential skills needed."

            4. **Experience Requirements**:
            - "How many years of experience are needed?"
            - "What type of previous experience is beneficial?"

            """ + skill1 + skill2 + skill3 + skill4

In [89]:
#Testing
#prompt = "[INST] Question: Who is Albert Einstein? \n Answer: [/INST]"
result = llm._call(prompt,user)
parsed_result = result['data']['content'] # type: ignore
parser.invoke(parsed_result)


"**Job Title and Summary**:\nJob Title: Senior Java Developer\nSummary: We are looking for a Senior Java Developer to join our team and be responsible for the development and maintenance of our Java-based applications. The ideal candidate should have a strong understanding of Core Java, Spring Boot, Hibernate, and Selenium.\n\n**Key Responsibilities**:\n- Design, develop, and maintain Java-based applications\n- Collaborate with cross-functional teams to define, design, and ship new features\n- Troubleshoot, debug, and upgrade existing systems\n- Write clean, efficient, and maintainable code\n- Conduct software analysis, programming, testing, and debugging\n- Participate in code reviews and provide constructive feedback\n- Document and maintain software functionality\n- Integrate software components and third-party programs\n- Verify and deploy programs and systems\n- Perform routine software maintenance\n\nDaily tasks involved:\n- Writing and reviewing code\n- Collaborating with team m

In [93]:
prompt2 = "Summarize it into 10 sentences keeping all the keeping all the key points and skills profiencies mentioned in the job description."
result2 = llm._call(prompt,user)
parsed_result2 = result2['data']['content'] # type: ignore
parser.invoke(parsed_result2)

"**Job Title and Summary**:\nJob Title: Senior Java Developer\nSummary: We are looking for a Senior Java Developer to join our team and be responsible for the development and maintenance of our Java applications. The ideal candidate will have a strong understanding of Core Java, Spring Boot, Hibernate, and Selenium.\n\n**Key Responsibilities**:\n- Design, develop, and maintain efficient, reusable, and reliable Java code\n- Collaborate with cross-functional teams to define, design, and ship new features\n- Identify bottlenecks and bugs, and devise solutions to these problems\n- Help maintain code quality, organization, and automatization\n- Daily tasks involve writing and reviewing code, debugging, and troubleshooting issues, and collaborating with team members to ensure project success\n\n**Qualifications and Skills**:\n- Bachelor's degree in Computer Science or a related field\n- Proven work experience as a Java Developer\n- Strong understanding of Core Java, Spring Boot, Hibernate, a